# Bài toán dự đoán khuynh hướng giá stock dựa trên model đã build trước đó.

Bài toán sử dụng là dùng text, price tăng hay giảm của gía cổ phiếu ngày hôm đó, từ đó mapping được độ tương quan giữa đoạn text và các cụm từ "giá chứng khoán tăng"
- label > 0.5 => Bài viết có xu hướng thiên về giá chứng khoán tăng.
- label < 0.5 => Bài viết có xu hướng thiên về giá chứng khoán giảm. 

VD:
    text| percentage_increase | text_compare
    "Fed tiếp tục chính sách nâng lãi suất" | 0.3 | "giá chứng khoán tăng"
    "Nhật nới lỏng chính sách tiền tệ" | 0.7 | "giá chứng khoán tăng"





In [2]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch


/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_88864/3777221515.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# load model pretrained
model = SentenceTransformer('./data/model_tendency')

In [4]:
from sentence_transformers import util

sent_test = [
    '1,4 tỷ dân Trung Quốc cũng không ở hết tất cả nhà bỏ hoang',
    'Quan chức Fed: Sẽ còn ít nhất 1 đợt nâng lãi suất',
    'Lệnh cấm xuất khẩu xăng dầu của Nga sẽ ảnh hưởng ra sao đến thị trường toàn cầu?',
    'Cổ phiếu chứng khoán, bất động sản “tan nát” dù xuất hiện dòng tiền bắt đáy',
    'Phiên giảm mạnh của chứng khoán và bất động diễn ra sau khi có thông tin NHNN bắt đầu rút lại thanh khoản trong nền kinh tế trong ngày 21/09. Ngoài ra, hai nhóm cổ phiếu mang nặng tính đầu cơ này đã leo dốc rất mạnh trong thời gian qua và là trụ cột chính cho đà tăng của thị trường',
    'Giá vàng thế giới hồi nhẹ, trong nước giữ mốc 69 triệu đồng/lượng',
]
t = model.encode(sent_test)
t1 = model.encode('chứng khoán tăng')
print('tendency', t.shape)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(t, t1)
print(cos_scores)

tendency (6, 768)
tensor([[0.5333],
        [0.5375],
        [0.4498],
        [0.4872],
        [0.4714],
        [0.4750]])


Chúng ta lấy data tin tức từ database để thực hiện việc dự đoán.

In [10]:
# reload package 

import pandas as pd
import regress_adapter
import importlib
importlib.reload(regress_adapter)

data = regress_adapter.get_news_date('2023-09-20')
print(data.head())

                                               title  \
0     Các huyện ngoại thành 'đua nhau' lên thành phố   
1     20/09: Đọc gì trước giờ giao dịch chứng khoán?   
2                Cổ phiếu Novaland bất ngờ tăng trần   
3  Công ty CP Vinam Land nói gì về việc huy động ...   
4  Các huyện ngoại thành 'đua nhau' lên thành phố...   

                                             content                date  
0  \n\n\n\n\n\n\n\n\n\n\n\n\nHà Mai\n-\nĐình Sơn\... 2023-09-20 06:33:00  
1  Cùng điểm lại những tin tức tài chính kinh tế ... 2023-09-20 06:21:51  
2  \n\n\n\n\nTIN MỚI\n\n\n\n\n  Mở cửa phiên giao... 2023-09-20 16:36:00  
3  \n\n\n\n\nTIN MỚI\n\n\n\n\nSáng 20-9, ông Nguy... 2023-09-20 07:45:00  
4  \n\n\n\n\n\n\n\n\n\n\n\n\nHà Mai\n-\nĐình Sơn\... 2023-09-21 06:35:00  


/Users/ngocp/Documents/projects/pyml/botapp/algorithm/regress_adapter.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def get_data(stock, date_start, date_end, export_url=None):


In [11]:
import torch

# We use cosine-similarity and torch.topk to find the highest 5 scores
def predict_relavency(sent = [], query = '', topk = 5):
    t = model.encode(sent)
    print('sentence embedding:', t[:2])
    t1 = model.encode(query)
    cos_scores = util.cos_sim(t, t1)
    print('cos_scores: với cụm từ trên:', cos_scores[:2])

    topk_values, topk_indices = torch.topk(cos_scores, k=topk, dim=0)
    return [(sent[i[0]], cos_scores[i]) for i in topk_indices]

data_test = (data['title'] + data['content']).values.tolist()

pred = predict_relavency(data_test, 'chứng khoán tăng', 5)
for (index, c) in enumerate(pred):
    print("câu thứ:" + str(index) + " :" + c[0][:100], c[1])

sentence embedding: [[ 0.07148299  0.05996135  0.02567843 ... -0.2907116   0.05045697
   0.02025556]
 [ 0.03870054  0.03300888  0.02753336 ... -0.27316827  0.09998763
  -0.09441629]]
cos_scores: với cụm từ trên: tensor([[0.4528],
        [0.4782]])
câu thứ:0 :Chứng khoán Đông Á - Thành viên của ngân hàng Đông Á, một trong những ngân hàng cổ phần lớn nhất nướ tensor([[0.5247]])
câu thứ:1 : Kinh tế Ukraine tăng trưởng gần 20%
Số liệu do Cơ quan Thống kê Nhà nước Ukraine mới công bố. Tăng  tensor([[0.5144]])
câu thứ:2 :Giá vàng hôm nay 20.9.2023: Vàng nhẫn tăng khoảng 4 triệu đồng/lượng







Thanh Xuân
-  xuanthanhn tensor([[0.5129]])
câu thứ:3 : FPT Telecom lọt top 10 doanh nghiệp số xuất sắc
Lễ vinh danh và trao chứng nhận "Top 10 doanh nghiệ tensor([[0.5042]])
câu thứ:4 :Chứng khoán Thành Công làm cổ đông lớn tại In Sách Giáo khoa Hòa PhátCTCP Chứng khoán Thành Công (UP tensor([[0.4996]])
